In [ ]:
# librerie
import os
import torch
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random
import collections
from tqdm import tqdm
import joblib 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision.utils import make_grid
from datetime import datetime

In [16]:
# riproducibilità
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# parametri principali
image_size = 224          # dimensione immagine (224x224)
batch_size = 128          # batch size per il DataLoader
encoding_dim = 256        # dimensione dello spazio latente dell'autoencoder
epochs = 30               # numero di epoche per l'allenamento
lr = 1e-3                 # learning rate per ottimizzatore
noise_std = 0.05          # rumore usato in fase di data augmentation
train_ratio, val_ratio = 0.7, 0.15  # percentuali per suddividere in train/val/test
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # utilizzo GPU se disponibile PROVA SENZA

pretty_labels = ['adenocarcinoma', 'beigno', 'carcinoma squamoso']  # nome delle classi più leggibili

In [ ]:
# percorsi utili
base_dir = os.getcwd()  # percorso base
data_dir = os.path.join(base_dir, 'data_histo') # percorso dataset
# percorso per salvare output
output_folder = os.path.join(base_dir, "ae_outputs")
os.makedirs(output_folder, exist_ok=True) # crea se non esiste
save_path = os.path.join(output_folder, f'ae_data_{encoding_dim}_{noise_std}.pth')
encoder_model_path = os.path.join(output_folder, f'ae_encoder_CNN_{encoding_dim}_{noise_std}.pt')

In [ ]:
# trasformazioni base (usate per val/test)
transform_base = transforms.Compose([
    transforms.Resize((image_size, image_size)), # ridimensiona le immagini
    transforms.ToTensor() # converte in tensore
])

In [ ]:
# caricamento e split
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform_base)  # carica tutte le immagini con le etichette
total_size = len(full_dataset)                                           # numero totale di immagini
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size                           # il resto va al test set

# suddivisione casuale ma riproducibile
train_indices, val_indices, test_indices = random_split(
    list(range(total_size)), [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

In [ ]:
train_subset = Subset(full_dataset, train_indices)
val_subset = Subset(full_dataset, val_indices)
test_subset = Subset(full_dataset, test_indices)

val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

In [ ]:
# caricamento dati augmentati (vedi file augmented_data.ipynb)
aug_data = torch.load(os.path.join("shared_augmented_data", f"augmented_train_data_{noise_std}.pt"))
train_imgs = aug_data['images']
train_labels = aug_data['labels']

# flatten immagini in vettori 1D
train_data = train_imgs.view(train_imgs.size(0), -1).numpy()

C:\Users\giken\AppData\Local\Temp\ipykernel_14480\4048884035.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  aug_data = torch.load(os.path.join("shared_augmented_data", 

In [ ]:
# estrazione e flattening di val e test
def extract_data(loader):
    data, labels = [], []
    for images, targets in tqdm(loader, desc="Estrazione dati"):
        flat = images.view(images.size(0), -1)
        data.append(flat)
        labels.append(targets)
    data = torch.cat(data, dim=0).numpy()
    labels = torch.cat(labels).numpy()
    return data, labels

val_data, val_labels = extract_data(val_loader)
test_data, test_labels = extract_data(test_loader)

Estrazione dati: 100%|██████████| 18/18 [00:16<00:00,  1.06it/s]


In [ ]:
# normalizzazione (Z-SCORE)
# la normalizzazione viene effettuata utilizzando la media e la deviazione standard 
# calcolate solo sul set di training. Questo è fondamentale per evitare data leakage: 
# usare informazioni statistiche dai dati di validazione o test potrebbe introdurre bias
# e compromettere la validità della valutazione del modello.
mean = train_data.mean(axis=0)
std = train_data.std(axis=0) + 1e-8  # evita divisione per zero

def zscore(data):
    return (data - mean) / std

train_data = zscore(train_data)
val_data = zscore(val_data)
test_data = zscore(test_data)

In [24]:
# autoencoder semplice per immagini 224x224 flattenate
input_dim = image_size * image_size * 3

In [ ]:
# definizione autoencoder convoluzionale
class ConvAutoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super().__init__()
        self.encoder_conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.fc_enc = nn.Linear(64 * 28 * 28, encoding_dim)
        self.fc_dec = nn.Linear(encoding_dim, 64 * 28 * 28)
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder_conv(x)
        z_flat = self.flatten(z)
        z_code = self.fc_enc(z_flat)
        z_up = self.fc_dec(z_code).view(-1, 64, 28, 28)
        x_recon = self.decoder_conv(z_up)
        return x_recon

    def encode_only(self, x):
        with torch.no_grad():
            z = self.encoder_conv(x)
            z_flat = self.flatten(z)
            z_code = self.fc_enc(z_flat)
        return z_code


In [ ]:
# costruzione modello
model = ConvAutoencoder(encoding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
# dataLoader per training AE
train_loader_ae = DataLoader(TensorDataset(train_imgs, train_labels), batch_size=batch_size, shuffle=True)

C:\Users\giken\AppData\Local\Temp\ipykernel_14480\3008790031.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor_dataset = TensorDataset(train_imgs.view(train_imgs.size(0), -1), torch.tensor(train_labels))


In [ ]:
# allenamento AE
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for x_batch, _ in train_loader_ae:
        x_batch = x_batch.to(device)
        optimizer.zero_grad()
        x_recon = model(x_batch)
        loss = criterion(x_recon, x_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss / len(train_loader_ae):.4f}")

Epoch 1/30 - Loss: 0.0673
Epoch 2/30 - Loss: 0.0512
Epoch 3/30 - Loss: 0.0462
Epoch 4/30 - Loss: 0.0423
Epoch 5/30 - Loss: 0.0391
Epoch 6/30 - Loss: 0.0370
Epoch 7/30 - Loss: 0.0357
Epoch 8/30 - Loss: 0.0350
Epoch 9/30 - Loss: 0.0344
Epoch 10/30 - Loss: 0.0338
Epoch 11/30 - Loss: 0.0329
Epoch 12/30 - Loss: 0.0324
Epoch 13/30 - Loss: 0.0316
Epoch 14/30 - Loss: 0.0309
Epoch 15/30 - Loss: 0.0303
Epoch 16/30 - Loss: 0.0303
Epoch 17/30 - Loss: 0.0296
Epoch 18/30 - Loss: 0.0294
Epoch 19/30 - Loss: 0.0292
Epoch 20/30 - Loss: 0.0291
Epoch 21/30 - Loss: 0.0289
Epoch 22/30 - Loss: 0.0286
Epoch 23/30 - Loss: 0.0283
Epoch 24/30 - Loss: 0.0280
Epoch 25/30 - Loss: 0.0278
Epoch 26/30 - Loss: 0.0276
Epoch 27/30 - Loss: 0.0278
Epoch 28/30 - Loss: 0.0275
Epoch 29/30 - Loss: 0.0275
Epoch 30/30 - Loss: 0.0271


In [29]:
# estrazione codifiche
def encode_dataset(data):
    model.eval()
    with torch.no_grad():
        data_tensor = torch.tensor(data, dtype=torch.float32).to(device)
        z = model.encoder(data_tensor).cpu().numpy()
    return z

train_encoded = encode_dataset(train_data)
val_encoded = encode_dataset(val_data)
test_encoded = encode_dataset(test_data)

In [30]:
# salvataggio risultati
torch.save({
    'train_data': train_encoded,
    'val_data': val_encoded,
    'test_data': test_encoded,
    'train_labels': train_labels,
    'val_labels': val_labels,
    'test_labels': test_labels,
    'class_names': pretty_labels
}, save_path)

torch.save(model.encoder.state_dict(), encoder_model_path)

print(f"Dati AE salvati in: {save_path}")
print(f"Encoder AE salvato in: {encoder_model_path}")

Dati AE salvati in: c:\Users\giken\Documents\Noemi\DAML-project\ae_outputs\ae_data_256_0.05.pth
Encoder AE salvato in: c:\Users\giken\Documents\Noemi\DAML-project\ae_outputs\ae_encoder_256_0.05.pt
